<div style="display: flex;">
    <span style="margin-left: auto; margin-right: 5px;  margin-top: 1px;"><a href="https://creativecommons.org/licenses/by/4.0/">
        <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
    </a></span>
    <span style="margin-right: auto; margin-left: 5px;"><a href="https://opensource.org/licenses/MIT">
        <img src="https://img.shields.io/badge/License-MIT-green.svg" />
    </a></span>
</div>

# Advanced pyBarSim examples

**Author:** Guillaume Rongier

In this notebook, we will look at different functionalities of pyBarSim to study the simulated deposits of wave-dominated shallow-marine environments in 2D.

### Imports

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cmocean
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.animation as animation
from IPython.display import HTML

from pybarsim import BarSim2D
from pybarsim.barsim import _sub_zero_extremity

### Setting

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 2**128

## 1. Deposition age

Let's start with a simple simulation with constant sea level and sediment supply:

In [ ]:
%%time

barsim = BarSim2D(np.linspace(0., -60., 250),
                  -5.,
                  20.,
                  spacing=150.,
                  max_wave_height_fair_weather=1.5,
                  max_wave_height_storm=6.,
                  sediment_size=(10., 80., 150., 250., 360.),
                  sediment_fraction=(0.35, 0.25, 0.2, 0.15, 0.05),
                  initial_substratum=(100., (0.35, 0.25, 0.2, 0.15, 0.05)),
                  seed=42)
barsim.run(10000., dt_fair_weather=10., dt_storm=1.)
barsim.finalize()
barsim.subsample(20)
barsim.finalize(on='subsequence');

And plot the final stratigraphy and mean grain size of the deposits:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, var='Mean grain size')
fig.colorbar(c[0], ax=ax, label=r'Mean grain size ($\mu$m)')
ax.set(xlabel='x (m)', ylabel='z (m)');

We need to expand the dimension of the variable `Time` before plotting the time of deposition, labelled `Age`:

In [ ]:
barsim.subsequence_['Age'] = barsim.subsequence_['Time'].expand_dims({'X': barsim.subsequence_.dims['X']}, axis=-1)

Now let's plot the result:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, var='Age', mask_zeros=False, sub_var='Mean grain size',
                            sub_zero_extremity=False, strip_zero_neighbors=True)
fig.colorbar(c[0], ax=ax, label=r'Deposition time (yr)')
ax.set(xlabel='x (m)', ylabel='z (m)');

## 2. Well simulation

Now let's simulate a more complex setting, with varying sea level and sediment supply in time but no storms:

In [ ]:
%%time

sea_level_curve = np.array([
    (0., -61.),
    (18000., -26.),
    (31000., -59.),
    (39000., -11.),
    (54000., -14.),
    (61000., -24.),
    (77000., -52.),
    (103000., -9.),
    (150000., -11.),
])
sediment_supply_curve = np.array([
    (0., 7.),
    (29000., 8.),
    (34000., 15.),
    (52000., 21.),
    (63000., 19.),
    (71000., 12.),
    (105000., 4.),
    (111000., 3.),
    (150000., 8.),
])
barsim = BarSim2D(np.linspace(0., -120., 300),
                  sea_level_curve,
                  sediment_supply_curve,
                  spacing=150.,
                  max_wave_height_fair_weather=1.5,
                  allow_storms=False,
                  sediment_size=(5., 50., 125., 250.),
                  sediment_fraction=(0.2, 0.3, 0.3, 0.2),
                  initial_substratum=(100., (0.2, 0.3, 0.3, 0.2)),
                  seed=42)
barsim.run(150000., dt_fair_weather=10.)
barsim.finalize()
barsim.subsample(20)
barsim.finalize(on='subsequence');

When plotting the well we'll need the water depth at the time of deposition:

In [ ]:
water_depth = barsim.sequence_['Sea level'] - barsim.sequence_['Elevation']
barsim.sequence_['Water depth'] = water_depth.where(water_depth > 0., 0.)

We can plot the final stratigraphy, this time with the location of the well (at the index *170* along the x axis):

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, var='Mean grain size')
fig.colorbar(c[0], ax=ax, label=r'Mean grain size ($\mu$m)')
ax.axvline(barsim.subsequence_['X'][170], c='red')
ax.text(barsim.subsequence_['X'][170], -110, 'Well location', ha='right', rotation=90, c='red')
ax.set(xlabel='x (m)', ylabel='z (m)');

And plot the well itself (at the index *170* along the x axis):

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 7))
p, sm = barsim.plot_well(ax, 170, on='sequence', var='Water depth', cmap=cmocean.cm.deep)
fig.colorbar(sm, ax=ax, pad=0.1, label='Water depth (m)')
ax.set(xlabel=r'Mean grain size ($\mu$m)', ylabel='z (m)');

Now let's do the same thing but on `subsequence_` instead of `sequence_` to see how the deposits are simplified:

In [ ]:
water_depth = barsim.subsequence_['Sea level'] - barsim.subsequence_['Elevation']
barsim.subsequence_['Water depth'] = water_depth.where(water_depth > 0., 0.)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 7))
p, sm = barsim.plot_well(ax, 170, on='subsequence', var='Water depth', cmap=cmocean.cm.deep,
                         linewidth=0.05, edgecolor='k')
fig.colorbar(sm, ax=ax, pad=0.1, label='Water depth (m)')
ax.set(xlabel=r'Mean grain size ($\mu$m)', ylabel='z (m)');

## 3. Sediment fractions as RGB

Let's keep the same setting but with storms and only 3 grain sizes:

In [ ]:
%%time

sea_level_curve = np.array([
    (0., -61.),
    (18000., -26.),
    (31000., -59.),
    (39000., -11.),
    (54000., -14.),
    (61000., -24.),
    (77000., -52.),
    (103000., -9.),
    (150000., -11.),
])
sediment_supply_curve = np.array([
    (0., 7.),
    (29000., 8.),
    (34000., 15.),
    (52000., 21.),
    (63000., 19.),
    (71000., 12.),
    (105000., 4.),
    (111000., 3.),
    (150000., 8.),
])
barsim = BarSim2D(np.linspace(0., -120., 300),
                  sea_level_curve,
                  sediment_supply_curve,
                  spacing=150.,
                  max_wave_height_fair_weather=1.5,
                  allow_storms=True,
                  sediment_size=(10., 75., 200.),
                  sediment_fraction=(0.3, 0.5, 0.2),
                  initial_substratum=(100., (0.3, 0.5, 0.2)),
                  seed=42)
barsim.run(150000., dt_fair_weather=10.)
barsim.finalize()
barsim.subsample(20)
barsim.finalize(on='subsequence');

We can plot the mean grain size as before:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, var='Mean grain size')
fig.colorbar(c[0], ax=ax, label=r'Mean grain size ($\mu$m)')
ax.set(xlabel='x (m)', ylabel='z (m)');

Or use the RGB channels to plot the fractions of each grain size. First we need to compute the fractions from the thickness of the deposits stored in `Deposits`:

In [ ]:
barsim.subsequence_['Fractions'] = barsim.subsequence_['Deposits'].copy()
barsim.subsequence_['Fractions'] /= barsim.subsequence_['Deposits'].values.sum(0)

Then we can plot the result:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
# This is based on what `plot_subsequence` does in the background
c = []
for i in range(1, barsim.subsequence_['Horizons'].shape[0]):
    rgba = []
    for j in range(3):
        layer = barsim.subsequence_['Fractions'].values[j, i, :-1]
        layer = _sub_zero_extremity(layer, layer)
        layer = np.tile(layer, (2, 1)).T
        rgba.append(layer)
    rgba.append(np.ones((len(barsim.subsequence_['X'].values[:-1]), 2)))
    rgba = np.stack(rgba, axis=-1)
    total = barsim.subsequence_['Fractions'].values[:, i, :-1].sum(0)
    rgba[np.isnan(total) | (total == 0.)] = 0.
    ci = ax.pcolormesh(np.tile(barsim.subsequence_['X'].values[:-1], (2, 1)).T,
                       barsim.subsequence_['Horizons'].values[i - 1:i + 1, :-1].T,
                       rgba,
                       shading='gouraud')
    c.append(ci)
ax.set(xlabel='x (m)', ylabel='z (m)')
# This is to add a rudimentary trianglular colorbar
axins = inset_axes(ax, width='100%', height='100%', loc='center',
                   bbox_to_anchor=(0.05, 0.2, 0.3, 0.3), bbox_transform=ax.transAxes)
#    Create a matrix with the colors
rgb = np.zeros((int(200*np.sqrt(3)/2.), 200, 3))
coords = np.stack(np.meshgrid(np.linspace(0., 1., 200), np.linspace(0., np.sqrt(3)/2., int(200*np.sqrt(3)/2.))), axis=-1)
angle = np.deg2rad(-90)
rotation_matrix = np.array([(np.cos(angle), -np.sin(angle)), (np.sin(angle), np.cos(angle))])
rgb[..., 0] = (coords@rotation_matrix)[..., 0]
rgb[..., 0] = (rgb[..., 0] - rgb[..., 0].min())/np.ptp(rgb[..., 0])
angle = np.deg2rad(30)
rotation_matrix = np.array([(np.cos(angle), -np.sin(angle)), (np.sin(angle), np.cos(angle))])
rgb[..., 1] = (coords@rotation_matrix)[..., 0]
rgb[..., 1] = (rgb[..., 1] - rgb[..., 1].min())/np.ptp(rgb[..., 1])
angle = np.deg2rad(-210)
rotation_matrix = np.array([(np.cos(angle), -np.sin(angle)), (np.sin(angle), np.cos(angle))])
rgb[..., 2] = (coords@rotation_matrix)[..., 0]
rgb[..., 2] = (rgb[..., 2] - rgb[..., 2].min())/np.ptp(rgb[..., 2])
#    Create a triangle to cut the matrix
triangle = plt.Polygon([(0., 0.), (1., 0.), (0.5, np.sqrt(3)/2.)], facecolor='none', edgecolor='k', linewidth=0.8)
#    Plot everything
axins.add_patch(triangle)
im = axins.imshow(rgb, extent=(0., 1., 0., np.sqrt(3)/2.))
im.set_clip_path(triangle)
axins.set_axis_off()
axins.text(0.5, np.sqrt(3)/2., '{0:g}'.format(barsim.subsequence_['Grain size'].values[0]) + r' $\mu$m', ha='center', va='bottom')
axins.text(1., 0., '{0:g}'.format(barsim.subsequence_['Grain size'].values[1]) + r' $\mu$m', ha='left', va='top')
axins.text(0., 0., '{0:g}'.format(barsim.subsequence_['Grain size'].values[2]) + r' $\mu$m', ha='right', va='top');

## 4. Pretty image and animation

Let's go back to the original grain sizes and run a longer simulation:

In [ ]:
%%time

sea_level_curve = np.array([
    (0., -71.),
    (18000., -33.),
    (31000., -66.),
    (39000., -18.),
    (54000., -21.),
    (61000., -31.),
    (77000., -55.),
    (93000., -14.),
    (117000., -8.),
    (121000., -3.),
    (129000., -2.),
    (153000., -7.),
    (158000., -3.),
    (182000., -4.),
    (191000., -34.),
    (200000., -5.),
])
sediment_supply_curve = np.array([
    (0., 7.),
    (29000., 8.),
    (34000., 15.),
    (52000., 21.),
    (63000., 19.),
    (71000., 12.),
    (105000., 1.),
    (111000., 2.),
    (137000., 3.),
    (152000., 17.),
    (158000., 15.),
    (165000., 16.),
    (200000., 11.),
])
barsim = BarSim2D(np.linspace(0., -120., 300),
                  sea_level_curve,
                  sediment_supply_curve,
                  spacing=150.,
                  max_wave_height_fair_weather=1.5,
                  max_wave_height_storm=6.,
                  sediment_size=(5., 50., 125., 250.),
                  sediment_fraction=(0.2, 0.3, 0.3, 0.2),
                  initial_substratum=(100., (0.2, 0.3, 0.3, 0.2)),
                  seed=42)
barsim.run(200000., dt_fair_weather=10., dt_storm=1.)
barsim.finalize()
barsim.subsample(20)
barsim.finalize(on='subsequence');

We can plot the stratigraphy using the same code as before:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, :-1],
                barsim.subsequence_['Horizons'][0, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, var='Mean grain size')
fig.colorbar(c[0], ax=ax, label=r'Mean grain size ($\mu$m)')
ax.set(xlabel='x (m)', ylabel='z (m)');

Or invest a bit more effort to make a prettier plot:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

plt.rc('font', family='Roboto Slab')

# Sea level
is_valid = barsim.subsequence_['Horizons'][0, :-1].values <= barsim.sequence_['Sea level'][-1].values
fill_sea = ax.fill_between(barsim.subsequence_['X'][:-1][is_valid],
                           barsim.subsequence_['Horizons'][0, :-1][is_valid],
                           barsim.sequence_['Sea level'][-1],
                           color='#c6dbef',
                           zorder=0)
# Substratum
fill_substratum = ax.fill_between(barsim.subsequence_['X'][:-1],
                                  barsim.subsequence_['Horizons'][0, :-1],
                                  barsim.subsequence_['Horizons'][0, :-1].min(),
                                  color='#f0f0f0')
# Deposits
c_deposits = barsim.plot_subsequence(ax)

ax.set(ylim=(None, 5.))
# Colorbar
axins = inset_axes(ax, width='100%', height='100%', loc='center',
                   bbox_to_anchor=(0.4, 0.205, 0.2, 0.035), bbox_transform=ax.transAxes)
fig.colorbar(c_deposits[0], cax=axins, orientation='horizontal', label=r'Mean grain size ($\mu$m)')
# Vertical scale bar
ax.plot([3000, 3000], [-80, -40], c='k', lw=2)
ax.text(3000 - 325, -60, '40 m', ha='right', va='center')
# Horizontal scale bar
ax.plot([4200, 14200], [-90, -90], c='k', lw=2)
ax.text(9200, -90 - 4, '10 km', ha='center', va='top')

ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_axis_off()

plt.show()

And save it:

In [ ]:
fig.savefig('./image.jpg', dpi=300, bbox_inches='tight', pad_inches=0)

Based on that plot, we can animate the evolution of the mean grain size of the deposits through time:

In [ ]:
%%time

fig, ax = plt.subplots(figsize=(12, 4))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

plt.rc('font', family='Roboto Slab')

# BarSim
barsim.run(1., dt_fair_weather=10., dt_storm=1.)
barsim.finalize()
barsim.subsample(20)
barsim.finalize(on='subsequence')
# Time
ax.annotate('Time:', (0.865, 0.95), va='baseline', xycoords='axes fraction')
label_time = ax.annotate('{:,}'.format(round(int(barsim.sequence_['Time'][-1]), -min(2, int(np.log10(barsim.sequence_['Time'][-1] + 1e-8))))).replace(',', r'$\,$') + ' yr',
                         (0.96, 0.95), ha='right', va='baseline', xycoords='axes fraction')
# Sea level
fill_sea = ax.fill_between(barsim.subsequence_['X'][:-1],
                           barsim.subsequence_['Horizons'][0, :-1].min(),
                           barsim.sequence_['Sea level'][-1],
                           color='#c6dbef',
                           zorder=0)
# Substratum
fill_substratum = ax.fill_between(barsim.subsequence_['X'][:-1],
                                  barsim.subsequence_['Horizons'][0, :-1].min(),
                                  barsim.subsequence_['Horizons'][0, :-1],
                                  color='#f0f0f0')
# Deposits
c_deposits = []
for i in range(1, barsim.subsequence_['Horizons'].shape[0]):
    layer = barsim.subsequence_['Mean grain size'][i, :-1].values
    layer = _sub_zero_extremity(layer, layer)
    layer = np.ma.masked_where(layer == 0., layer)
    ci = ax.pcolormesh(np.tile(barsim.subsequence_['X'][:-1].values, (2, 1)).T,
                       barsim.subsequence_['Horizons'][i - 1:i + 1, :-1].T,
                       np.tile(layer, (2, 1)).T,
                       shading='gouraud',
                       vmin=0.,
                       vmax=240.)
    c_deposits.append(ci)

ax.set(xlim=(barsim.subsequence_['X'][0], barsim.subsequence_['X'][-2]), ylim=(barsim.subsequence_['Horizons'][0, :-1].min(), 8.))
# Colorbar
axins = inset_axes(ax, width='100%', height='100%', loc='center',
                   bbox_to_anchor=(0.4, 0.205, 0.18, 0.03), bbox_transform=ax.transAxes)
fig.colorbar(c_deposits[0], cax=axins, orientation='horizontal', label=r'Mean grain size ($\mu$m)')
# Vertical scale bar
ax.plot([2750, 2750], [-80, -40], c='k', lw=2)
ax.text(2750 - 325, -60, '40 m', ha='right', va='center')
# Horizontal scale bar
ax.plot([4000, 14000], [-90, -90], c='k', lw=2)
ax.text(9000, -90 - 3.5, '10 km', ha='center', va='top')

ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_axis_off()

plt.close()


def update(t):
    global c_deposits
    # BarSim
    barsim.run(t, dt_fair_weather=10., dt_storm=1.)
    barsim.finalize()
    barsim.subsample(20)
    barsim.finalize(on='subsequence')
    # Time
    label_time.set_text('{:,}'.format(round(int(barsim.sequence_['Time'][-1]), -min(2, int(np.log10(barsim.sequence_['Time'][-1] + 1e-8))))).replace(',', r'$\,$') + ' yr')
    # Sea level
    path = fill_sea.get_paths()[0]
    path.vertices[len(barsim.sequence_['X'][:-1]) + 2:-1, 1] = barsim.sequence_['Sea level'][-1].values
    # Substratum
    path = fill_substratum.get_paths()[0]
    path.vertices[len(barsim.sequence_['X'][:-1]) + 2:-1, 1] = barsim.subsequence_['Horizons'][0, :-1].values[::-1]
    # Deposits
    for i in range(1, barsim.subsequence_['Horizons'].shape[0]):
        layer = barsim.subsequence_['Mean grain size'][i, :-1].values
        layer = _sub_zero_extremity(layer, layer)
        layer = np.ma.masked_where(layer == 0., layer)
        if i - 1 < len(c_deposits):
            c_deposits[i - 1].set_array(np.tile(layer, (2, 1)).T)
            c_deposits[i - 1]._coordinates[..., 1] = barsim.subsequence_['Horizons'][i - 1:i + 1, :-1].T
        else:
            ci = ax.pcolormesh(np.tile(barsim.subsequence_['X'][:-1].values, (2, 1)).T,
                               barsim.subsequence_['Horizons'][i - 1:i + 1, :-1].T,
                               np.tile(layer, (2, 1)).T,
                               shading='gouraud',
                               vmin=0.,
                               vmax=240.)
            c_deposits.append(ci)

    return label_time, fill_sea, fill_substratum, *c_deposits

ani = animation.FuncAnimation(fig, update, range(0, 200100, 100), interval=30, blit=True)
# To visualize the animation directly in the notebook, uncomment this line.
# It takes a long time to run, and there seems to be a bug if `ani` is
# displayed and saved (last frame is displayed statically).
# HTML(ani.to_jshtml())

<div class="alert alert-block alert-warning">
<b>&#9888;</b> The code above isn't optimal so the next cell takes a lot of time to run (several hours).
</div>

And save the animation:

In [ ]:
%%time

writer = animation.FFMpegWriter(fps=25)
ani.save('./pybarsim_animation.mp4', writer=writer, dpi=300)